In [1]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 62.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 70.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.2 MB/s eta 0:00:0

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, Dropout
from keras.models import Sequential

from sklearn.decomposition import PCA

from sklearn.neural_network import MLPRegressor

import numpy as np

import xgboost as xgb

In [3]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [4]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [6]:
df["concatenated"] = df['Question'] + " " + df['Student Answer']

In [7]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["concatenated"]], df["Grade"], test_size=0.1, random_state=42)

In [8]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [9]:
train_X["concatenated"] = train_X["concatenated"].apply(extract_pos).apply(process_dataset)

test_X["concatenated"] = test_X["concatenated"].apply(extract_pos).apply(process_dataset)

In [10]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 2)
(2197, 2)


In [11]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]

In [12]:
sentences = list(train_X["concatenated"])

In [13]:
rm -rf ~/.cache/huggingface/transformers/bert-base-uncased

In [14]:
from flair.embeddings import FlairEmbeddings
from flair.data import Sentence
import flair

In [15]:
flair_forward = flair.embeddings.FlairEmbeddings('multi-forward')
flair_backward = flair.embeddings.FlairEmbeddings('multi-backward')
flair_en_forward = flair.embeddings.FlairEmbeddings('en-forward')

2023-06-08 16:02:25,071 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-forward-v0.1.pt not found in cache, downloading to /tmp/tmphvgcmig1


100%|██████████| 165M/165M [00:08<00:00, 20.6MB/s]

2023-06-08 16:02:33,901 copying /tmp/tmphvgcmig1 to cache at /root/.flair/embeddings/lm-jw300-forward-v0.1.pt


2023-06-08 16:02:34,414 removing temp file /tmp/tmphvgcmig1
2023-06-08 16:02:46,792 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-backward-v0.1.pt not found in cache, downloading to /tmp/tmp8p0v6i4i


100%|██████████| 165M/165M [00:08<00:00, 20.5MB/s]

2023-06-08 16:02:55,585 copying /tmp/tmp8p0v6i4i to cache at /root/.flair/embeddings/lm-jw300-backward-v0.1.pt


2023-06-08 16:02:55,790 removing temp file /tmp/tmp8p0v6i4i
2023-06-08 16:02:56,695 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmporl_amaw


100%|██████████| 69.7M/69.7M [00:04<00:00, 18.1MB/s]

2023-06-08 16:03:01,141 copying /tmp/tmporl_amaw to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2023-06-08 16:03:01,221 removing temp file /tmp/tmporl_amaw


In [16]:
flair_news_forward = flair.embeddings.FlairEmbeddings('news-forward')
flair_news_backward = flair.embeddings.FlairEmbeddings('news-backward')

2023-06-08 16:03:02,080 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmposetc2aj


100%|██████████| 69.7M/69.7M [00:04<00:00, 18.2MB/s]

2023-06-08 16:03:06,493 copying /tmp/tmposetc2aj to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2023-06-08 16:03:06,611 removing temp file /tmp/tmposetc2aj


In [96]:
train_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":train_X["concatenated"]})
test_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":test_X["concatenated"]})

In [97]:
def get_flair_multi_forward_embeddings(text):
  sentence = Sentence(text)
  flair_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [98]:
def get_flair_multi_backward_embeddings(text):
  sentence = Sentence(text)
  flair_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [99]:
def get_flair_news_forward_embeddings(text):
  sentence = Sentence(text)
  flair_news_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [100]:
def get_flair_news_backward_embeddings(text):
  sentence = Sentence(text)
  flair_news_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [101]:
def get_flair_en_forward_embeddings(text):
  sentence = Sentence(text)
  flair_en_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [102]:
train_Embeddings_df["Q_Student_Answer_Embeddings"] = train_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_flair_multi_backward_embeddings)

test_Embeddings_df["Q_Student_Answer_Embeddings"] = test_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_flair_multi_backward_embeddings)

In [103]:
train_Embeddings_df

,Q_Student_Answer_Embeddings
199,"[-0.005689676, 9.5152514e-05, -0.007687468, -0..."
229,"[0.024751464, 0.002177488, -0.0116156265, -0.0..."
521,"[0.011926344, -7.440133e-06, -0.014075156, -0...."
651,"[0.006801787, 0.0011593327, -0.019843832, -0.0..."
450,"[0.0016229724, 0.00028979598, -0.01107227, -0...."
...,...
1638,"[0.003310446, 0.0002718546, -0.031109465, 4.30..."
1095,"[0.012538168, 0.00048947515, -0.007584193, -0...."
1130,"[0.0127403205, 0.00027708797, -0.02227142, -0...."
1294,"[0.002830763, 0.001259419, -0.011745819, -0.00..."


In [104]:
train_X["QSA_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Q_Student_Answer_Embeddings"], axis = 1)
test_X["QSA_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Q_Student_Answer_Embeddings"], axis = 1)

In [105]:
train_df = pd.DataFrame(list(train_X["QSA_Embeddings"]))
test_df = pd.DataFrame(list(test_X["QSA_Embeddings"]))

In [106]:
train_df.to_csv("flair_train_embeddings_QSA.csv")
test_df.to_csv("flair_test_embeddings_QSA.csv")

In [107]:
pca = PCA(n_components=100)  # Specify the number of components you want to keep
train_df_pca = pca.fit_transform(train_df)

In [108]:
test_df_pca = pca.transform(test_df)

In [109]:
ncols = train_df_pca.shape[1]

In [110]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(ncols,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_df_pca, train_y, epochs=10, batch_size=32, verbose=1)

y_true = test_y
y_preds = model.predict(test_df_pca)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
cnn_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
cnn_mse = mean_squared_error(y_true, np.round(y_preds, 1))
cnn_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
cnn_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
cnn_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(cnn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(cnn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(cnn_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(cnn_mape, 2))
print("R2 Score: ", np.round(cnn_r2, 2))

Epoch 1/10
69/69 [==============================] - 1s 3ms/step - loss: 4.2214
Epoch 2/10
69/69 [==============================] - 0s 3ms/step - loss: 2.7302
Epoch 3/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6272
Epoch 4/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5832
Epoch 5/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5941
Epoch 6/10
69/69 [==============================] - 0s 4ms/step - loss: 2.5445
Epoch 7/10
69/69 [==============================] - 0s 5ms/step - loss: 2.5464
Epoch 8/10
69/69 [==============================] - 0s 5ms/step - loss: 2.5693
Epoch 9/10
69/69 [==============================] - 0s 4ms/step - loss: 2.6115
Epoch 10/10
8/8 [==============================] - 0s 3ms/step
Mean Absolute Error (MAE):  1.23
Mean Squared Error (MSE):  2.04
Root Mean Squared Error (RMSE):  1.43
Mean Absolute Percentage Error (MAPE):  524100887339309.2
R2 Score:  -0.03


In [111]:
# Create an MLP regressor with a single linear output layer
mlp = MLPRegressor(hidden_layer_sizes=(100,50,25,1), activation='identity', solver='lbfgs')

# Train the MLP regressor
mlp.fit(train_df_pca, train_y)

y_preds = mlp.predict(test_df_pca)

y_true = test_y

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
mlp_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
mlp_mse = mean_squared_error(y_true, np.round(y_preds, 1))
mlp_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
mlp_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
mlp_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(mlp_mae, 2))
print("Mean Squared Error (MSE): ", np.round(mlp_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(mlp_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(mlp_mape, 2))
print("R2 Score: ", np.round(mlp_r2, 2))

Mean Absolute Error (MAE):  1.13
Mean Squared Error (MSE):  1.91
Root Mean Squared Error (RMSE):  1.38
Mean Absolute Percentage Error (MAPE):  554597421688499.6
R2 Score:  0.04


In [112]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df_pca, train_y)

preds = LR_model.predict(test_df_pca)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  1.15
Mean Squared Error (MSE):  1.94
Root Mean Squared Error (RMSE):  1.39
Mean Absolute Percentage Error (MAPE):  556975791965096.56
R2 Score:  0.03


In [113]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df_pca, train_y)  #fit the model
    pred=knn_model.predict(test_df_pca) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

y_true = test_y
knn_preds = knn_model.predict(test_df_pca)

min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.16
Mean Squared Error (MSE):  2.63
Root Mean Squared Error (RMSE):  1.62
R2 Score:  -0.31


In [114]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.36
Mean Squared Error (MSE):  2.37
Root Mean Squared Error (RMSE):  1.54
Mean Absolute Percentage Error (MAPE):  501829672764141.25
R2 Score:  -0.19


In [115]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df_pca, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df_pca))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.13
Mean Squared Error (MSE):  1.9
Root Mean Squared Error (RMSE):  1.38
Mean Absolute Percentage Error (MAPE):  534917343495842.9
R2 Score:  0.04


In [116]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df_pca, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df_pca))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.18
Mean Squared Error (MSE):  2.35
Root Mean Squared Error (RMSE):  1.53
Mean Absolute Percentage Error (MAPE):  527564515506351.8
R2 Score:  -0.18
